In [5]:
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

In [6]:
train_data_gen = ImageDataGenerator(rescale=1.0/255.0)
validation_data_gen = ImageDataGenerator(rescale=1.0/255.0)
print(type(train_data_gen))

<class 'tensorflow.python.keras.preprocessing.image.ImageDataGenerator'>


In [7]:
train_generator = train_data_gen.flow_from_directory(
    'data/train',
    target_size=(48,48),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical"
)

validation_generator = validation_data_gen.flow_from_directory(
    'data/test',
    target_size=(48,48),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical"
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [8]:
model = Sequential([
    Conv2D(32,(3,3),activation='relu',input_shape=(48,48,1)),
    MaxPooling2D((2,2)),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.3),

    Conv2D(128,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(128,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.3),

    Flatten(),
    Dense(512,activation='relu'),
    Dropout(0.2),
    Dense(7,activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

em_model = model.fit_generator(
    train_generator,
    steps_per_epoch = 28709//64,
    epochs = 50,
    validation_data = validation_generator,
    validation_steps = 7178//64
)

print(model.summary)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
448/448 [==============================] - 65s 146ms/step - loss: 1.8059 - accuracy: 0.2515 - val_loss: 1.7214 - val_accuracy: 0.2944
Epoch 2/50
448/448 [==============================] - 37s 82ms/step - loss: 1.6023 - accuracy: 0.3650 - val_loss: 1.4187 - val_accuracy: 0.4492
Epoch 3/50
448/448 [==============================] - 34s 76ms/step - loss: 1.4173 - accuracy: 0.4493 - val_loss: 1.3145 - val_accuracy: 0.4964
Epoch 4/50
448/448 [==============================] - 34s 75ms/step - loss: 1.3351 - accuracy: 0.4886 - val_loss: 1.2976 - val_accuracy: 0.5045
Epoch 5/50
448/448 [==============================] - 35s 79ms/step - loss: 1.2791 - accuracy: 0.5094 - val_loss: 1.2288 - val_accuracy: 0.5271
Epoch 6/50
448/448 [==============================] - 34s 75ms/step - loss: 1.2369 - accuracy: 0.5280 - val_loss: 1.1909 - val_accuracy: 0.5384
Epoch 7/50
448/448 [==============================] - 35s 7

In [11]:
model_json = model.to_json()
with open("emotion_model.json","w") as json_file:
    json_file.write(model_json)

model.save_weights('emotion_model.h5')